# Translate any video into any language

## Preparation 

In [1]:
# imports 
# this might be necessary if you are using docker the way I am
import os 
os.chdir('video-translation')
from utils import *

In [2]:
# set url for video
url = "https://www.youtube.com/watch?v=a03U45jFxOI"

## Process Audio

In [3]:
# download audio
video_file = download_video(url, output_dir='cache')

[youtube] Extracting URL: https://www.youtube.com/watch?v=a03U45jFxOI
[youtube] a03U45jFxOI: Downloading webpage
[youtube] a03U45jFxOI: Downloading ios player API JSON
[youtube] a03U45jFxOI: Downloading mweb player API JSON
[youtube] a03U45jFxOI: Downloading m3u8 information
[info] a03U45jFxOI: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 63
[download] Destination: cache/a03U45jFxOI.f616.mp4
[download]  44.3% of ~ 104.92MiB at    4.55MiB/s ETA 00:13 (frag 29/63)

[download] Got error: [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)


[download] fragment not found; Skipping fragment 30 ...
[download] 100% of  108.79MiB in 00:00:34 at 3.18MiB/s                   
[download] Destination: cache/a03U45jFxOI.f251.webm
[download] 100% of    4.48MiB in 00:00:00 at 8.60MiB/s   
[Merger] Merging formats into "cache/a03U45jFxOI.webm"
Deleting original file cache/a03U45jFxOI.f616.mp4 (pass -k to keep)
Deleting original file cache/a03U45jFxOI.f251.webm (pass -k to keep)


In [4]:
# transcribe audio
transcription = transcribe_audio(video_file)
start, end, english = extract_transcription(transcription)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
 98%|████████

## Translate Trasncript

In [5]:
# summarize transcription
# you might want to generate a long context Ollama model first
summarized_transcription = summarize_transcription(transcription, "8k-qwen2.5:7b")

In [6]:
# translate to any langugaes
to_lang = "Simpliefied Chinese (Mandarin)"
system_prompt = f"""
You are a multilingual subtitle translation assistant. Translate the following English subtitle to {to_lang}. 
Here is a short summary of the video content: {summarized_transcription}
Ensure the translation feels natural and culturally localized, avoiding direct English phrasing where possible.
Each translation should consider the preceding sentence to ensure contextual accuracy. 
Reply only the translation text and nothing else.
""" 

chinese = translate_subtitle(english, system_prompt, "qwen2.5:7b")

100%|██████████| 86/86 [00:37<00:00,  2.32it/s]


## Save to SRT

In [7]:
# save srt files
filename = "".join(video_file.split('.')[:-1])

save_srt(f'{filename}.en.srt', start, end, english)
save_srt(f'{filename}.zh.srt', start, end, chinese)

In [8]:
# after manual correction, update the srt files
# start, end, english = load_srt(f'{filename}.en.srt')
start, end, chinese = load_srt(f'{filename}.zh.srt')

save_srt(f'{filename}.davinci.zh.srt', start, end, chinese, True)
# save_srt(f'{filename}.davinci.en.srt', start, end, english, True)
# save_bilingual_srt(f'{filename}.zh.en.srt', start, end, english, chinese)
# save_bilingual_srt(f'{filename}.davinci.zh.en.srt', start, end, chinese, english, True)